In [1]:
import nltk
!pip install contractions
!pip install pyspellchecker
!pip install datasets




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requ

In [2]:
import pandas as pd
import nltk
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import contractions  # Import the contractions library
from spellchecker import SpellChecker  # Import the SpellChecker library
from nltk import pos_tag
from nltk.corpus import wordnet #import wordnet for POS tagging
from datasets import load_dataset
import spacy
import shutil
import os

In [3]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('omw-1.4')  # Optional but useful for lemmatization
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.data.find('taggers/averaged_perceptron_tagger.zip')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


ZipFilePathPointer('/root/nltk_data/taggers/averaged_perceptron_tagger.zip', '')

In [30]:
# Load stopwords once
stop_words = set(stopwords.words('english'))

# Initialize the spellchecker
spell = SpellChecker()

# Load spaCy English model ✅
nlp = spacy.load("en_core_web_sm")

def expand_clitics(text):
    """Expand contractions like "I'm" → "I am" """
    return contractions.fix(text)

def correct_spelling(tokens):
    """Spell check each word"""
    return [spell.correction(word) if spell.correction(word) else word for word in tokens]

def clean_text(text):
    """Convert text to lowercase, expand contractions, and remove punctuation"""
    text = text.lower()
    text = expand_clitics(text)
    text = re.sub(f"[{string.punctuation}]", "", text)
    return text

def tokenize_text(text):
    """Tokenize text"""
    return word_tokenize(text)

def lemmatize_text(text):
    """Perform lemmatization using spaCy with accurate POS tagging"""
    doc = nlp(text)
    return [token.lemma_ for token in doc]

def load_slang_mapping(file_path):
    """Load the slang mapping from the provided CSV file"""
    slang_df = pd.read_csv(file_path, usecols=[1, 2], names=['acronym', 'expansion'], skiprows=1)
    slang_mapping = dict(zip(slang_df['acronym'].str.lower(), slang_df['expansion'].str.lower()))
    return slang_mapping

def replace_slang(tokens, slang_mapping):
    """Replace slang words with their expansions"""
    doc = nlp(' '.join(tokens))
    updated_tokens = []

    for token in doc:
        if token.text.lower() == 'am' and token.pos_ == 'AUX':
            updated_tokens.append(token.text)
        elif token.text.lower() in slang_mapping:
            expansion_tokens = [t.text for t in nlp(slang_mapping[token.text.lower()])]
            updated_tokens.extend(expansion_tokens)
        else:
            updated_tokens.append(token.text)

    return updated_tokens

def get_pos_tags(tokens):
    """Get POS tags for each token using spaCy"""
    doc = nlp(' '.join(tokens))
    return [(token.text, token.pos_) for token in doc]

def preprocess_data(file_path, slang_file_path):
    """Main preprocessing function"""
    df = pd.read_csv(file_path)
    df.dropna(inplace=True)

    stemmer = PorterStemmer()

    # Load slang mapping
    slang_mapping = load_slang_mapping(slang_file_path)

    # Clean and process text
    df['cleaned_text'] = df['Utterance'].apply(clean_text)
    df['tokenized_text'] = df['cleaned_text'].apply(tokenize_text)

    # Replace slang words
    df['slang_replaced_text'] = df['tokenized_text'].apply(lambda tokens: replace_slang(tokens, slang_mapping))

    # Apply spell checking
    df['spell_checked_text'] = df['slang_replaced_text'].apply(correct_spelling)

    # Remove stopwords
    df['filtered_text'] = df['spell_checked_text'].apply(lambda tokens: [word for word in tokens if word not in stop_words])

    # Apply stemming
    df['stemmed_text'] = df['filtered_text'].apply(lambda tokens: [stemmer.stem(word) for word in tokens])

    # Apply lemmatization using spaCy (POS tagging)
    df['lemmatized_text'] = df['cleaned_text'].apply(lemmatize_text)

     # Get POS tags for each tokenized text
    df['pos_tags'] = df['slang_replaced_text'].apply(lambda tokens: get_pos_tags(tokens))

    return df


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/local/lib/python3.11/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [6]:
# Mount Google Drive (only need to do this once per session)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
# Specify file paths in Google Drive
file_path = '/content/drive/MyDrive/BTECH_PROJECT/abc.csv'  # Update path as necessary
output_path = '/content/drive/MyDrive/BTECH_PROJECT/preprocessed_data_abc.csv'  # Update path as necessary
slang_file_path = '/content/drive/MyDrive/BTECH_PROJECT/slang.csv'
# Run preprocessing
df_preprocessed = preprocess_data(file_path, slang_file_path)

# Save and display results
df_preprocessed.to_csv(output_path, index=False)
print(df_preprocessed[['Utterance', 'cleaned_text', 'tokenized_text','slang_replaced_text',  'spell_checked_text', 'lemmatized_text', 'pos_tags']].head())  # Display the original, cleaned, tokenized, and spell-checked columns

                                           Utterance  \
0                                 Are u doing today?   
1    Alright, thanks. How are you? What is the time?   
2  I'm doing okay. Thx for asking. It is 10 am. I...   
3                                         Yeah, brb    
4                               Can lmk about those?   

                                        cleaned_text  \
0                                are you doing today   
1        alright thanks how are you what is the time   
2  i am doing okay thanks for asking it is 10 am ...   
3                                          yeah brb    
4                                can lmk about those   

                                      tokenized_text  \
0                           [are, you, doing, today]   
1  [alright, thanks, how, are, you, what, is, the...   
2  [i, am, doing, okay, thanks, for, asking, it, ...   
3                                        [yeah, brb]   
4                           [can, lmk, about, 